### Scraping (GetMatch)

In [ ]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

Ссылка страницы в списке вакансий имеет вид ```https://getmatch.ru/vacancies?p={page}```, при этом если ввести слишком большой номер страницы, она отображается корректно, просто без вакансий. Поэтому, чтобы собрать все ссылки, будем идти по страницам до тех пор, пока на них есть вакансии.

In [ ]:
def get_page_vacancies_urls(page):
    page_url = f'https://getmatch.ru/vacancies?p={page}'

    page = requests.get(page_url)
    if page.status_code != 200:
        return

    page_vacancies_urls = []

    soup = BeautifulSoup(page.content, 'html.parser')

    for link in soup.find_all('a'):
        if '?s=offers' in link.get('href') and link.get('href').startswith('/vacancies'):
            page_vacancies_urls.append(link.get('href'))

    return page_vacancies_urls

In [ ]:
vacancies_urls = []
page = 1

while True:
    page_vacancies_urls = get_page_vacancies_urls(page)

    if not page_vacancies_urls:
        break

    vacancies_urls.extend(page_vacancies_urls)
    page += 1

print(vacancies_urls)

Проверим, что собрали. Действительно, на момент выполнения кода на сайте было 1203 вакансий и 2 рекламные страницы (которые имеют такую же ссылку, как вакансии). Итого 1205.

In [ ]:
len(vacancies_urls)

Изучим полученную разметку на примере одной ссылки. Напишем функцию для сбора информации по конкретной вакансии.

In [ ]:
vacancy = requests.get(f'https://getmatch.ru{vacancies_urls[6]}')
soup = BeautifulSoup(vacancy.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html dir="ltr" lang="ru">
 <head>
  <link crossorigin="" href="https://fonts.gstatic.com" rel="preconnect"/>
  <meta charset="utf-8"/>
  <title>
   Вакансия Affiliate Manager, работа в Winline, в Москве — getmatch
  </title>
  <base href="/"/>
  <meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
  <meta content="#ffffff" name="theme-color"/>
  <link href="/uploads/favicons/favicon.svg" rel="icon"/>
  <link color="#205AED" href="/uploads/favicons/mask-icon.svg" rel="mask-icon"/>
  <link href="/uploads/favicons/apple-touch-icon.png" rel="apple-touch-icon"/>
  <link href="/uploads/favicons/manifest.json" rel="manifest"/>
  <link href="/uploads/favicons/favicon.ico" rel="shortcut icon"/>
  <style type="text/css">
   @font-face{font-family:'Montserrat';font-style:normal;font-weight:300;font-display:swap;src:url(https://fonts.gstatic.com/s/montserrat/v29/JTUSjIg1_i6t8kCHKm459WRhyyTh89ZNpQ.woff2) format('woff2');unicode-range:U+0460-052F, U

In [ ]:
def get_info(soup):
    info = {}

    # название вакансии и компании
    info['title'] = soup.find('h1').text.strip()
    info['company'] = soup.find('h2').find('a').text.strip()

    # зарплата
    salary_container = soup.find(lambda tag: tag.name == 'h3' and any(char in tag.get_text() for char in ['₽', '$', '€']))

    if salary_container:
      info['salary'] = salary_container.text.strip()

    # места и типы работы
    place_container = soup.find('div', {'class': 'b-vacancy-locations'})

    if place_container:
      locations = place_container.find_all('span', {'class': 'g-label-secondary'})

      if locations:
        info['locations'] = [item.text.strip() for item in locations]

      categories = place_container.find_all('span', {'class': 'g-label-zanah'})

      if categories:
          info['categories'] = [item.text.strip() for item in categories]

    # основные условия
    terms_container = soup.find('div', {'class': 'b-specs'})

    if terms_container:
      term_titles = terms_container.find_all('div', {'class': 'b-term'})

      if term_titles:
        terms = {}

        for term_title in term_titles:
          term_value = term_title.find_next('div', {'class': 'b-value'})

          if term_value:
            terms[term_title.text.strip()] = term_value.text.strip()

        info['terms'] = terms

    # стек технологий
    stack_container = soup.find('div', {'class': 'b-vacancy-stack-container'})

    if stack_container:
      stack = stack_container.find_all('span')
      info['stack'] = [item.text.strip() for item in stack]

    # описание
    description_container = soup.find('section', {'class': 'b-vacancy-description'})

    if description_container:
      description_titles = description_container.find_all('h2') + description_container.find_all('h3')

      if description_titles:
        descriptions = {}

        for description_title in description_titles:
          description_list = description_title.find_next('ul')

          if not description_list:
            continue

          description_bullets = description_list.find_all('li')


          if description_bullets:
            description_value = ''

            for description_bullet in description_bullets:
              description_value += description_bullet.text.strip() + ' '

            if len(description_value) != 0:
              description_value = description_value[:-1]
              descriptions[description_title.text.strip()] = description_value

        info['descriptions'] = descriptions

    return info

Посмотрим, как выглядит получившийся словарь. Все соответсвует ожиданиям :)

In [ ]:
get_info(soup)

{'title': 'Affiliate Manager',
 'company': 'Winline',
 'salary': '150 000 —\u200d 200 000 ₽/мес на руки',
 'locations': ['📍 Москва (м. Шелепиха)'],
 'categories': ['Офис несколько дней в неделю'],
 'terms': {'Специализация': 'Product Analyst / Product Manager / Project Manager',
  'Уровень': 'Middle',
  'Требуемый опыт': '2+ лет'},
 'stack': ['Google Analytics', 'Яндекс.Метрика', 'Excel', 'PowerPoint'],
 'descriptions': {'Обязанности': 'Поддерживать и развивать партнерские отношения с текущими веб-мастерами. Искать новых целевых клиентов. Вести отчетность и анализировать текущий трафик. Контролировать размещение промо-материалов на сайтах СМИ. Проводить сверку и еженедельный контроль партнеров на предмет фрода. Оперативно коммуницировать с партнерами в чатах и по почте. Отслеживать тренды рынка и мониторить конкурентов для оперативного реагирования и изменений.',
  'Требования': 'Опыт работы в Affiliate-маркетинге на позиции менеджер (от 2 лет). Понимание видов трафика и принципов их р

Соберем информацию о вакансиях по каждой из полученных ранее ссылок (пропустим первые три, как и было замечено ранее, это реклама: weekend offer и прочее).

In [ ]:
vacancies = []

for url in vacancies_urls[2:]:
    vacancy = requests.get(f'https://getmatch.ru{url}')
    soup = BeautifulSoup(vacancy.content, 'html.parser')

    info = get_info(soup)
    vacancies.append(info)

print(vacancies)

Посмотрим какие есть значения у разных ключей.

In [ ]:
unique_titles = set(vacancy.get('title') for vacancy in vacancies)

print('\n'.join(unique_titles))

Content Product Manager
Бизнес партнер по информационной безопасности
Руководитель команды аналитики маркетинга (Работа)
Тимлид разработки DWH
Тимлид команды продуктовой аналитики
Старший аналитик
Talent Acquisition Researcher
Руководитель направления по аналитике (Лояльность)
Ведущий аналитик 1С
Руководитель инфраструктурной команды
Разработчик на Golang/Python (Платёжный шлюз)
Специалист сопровождения АБС (Финтех-сервисы)
DevOps (Финтех)
Middle/Senior Product Designer
Marketing Analyst
Инженер доступности сервисов / SRE (Cloud Storage)
QA Automation Engineer
Сетевой инженер
Node.js Developer (Парсинг)
Infrastructure Engineering Lead / Менеджер ИТ-инфраструктуры
Senior Frontend Developer (React)
Senior Golang Developer (Продуктовая платформа)
Младший инженер технической поддержки пользователей
Системный администратор технической поддержки
DevSecOps & Security Engineering Team Lead
Security Champion in SW Development / Специалист по информационной безопасности (TATLIN.OBJECT)
Product A

In [ ]:
unique_companies = set(vacancy.get('company') for vacancy in vacancies)

print('\n'.join(unique_companies))

Альфа-банк
ОТП Банк
JTI
Collectly
М.Видео
Dodo Brands
Сравни
Pari
Flocktory
Яндекс MultiTrack
Деметра Холдинг
Spectral::Technologies
Galileosky
CV Recruitment
Payler
Плюс Фантех
Axiom JDK
Aston
Kuvelli
Inita
Selectel
=nil; Foundation
TradingView
Navio
Okko
Банк Точка
Just AI
Название скрыто
Rubbles
Делимобиль
Softline
Sportmaster Lab
Газпром нефть
Лига Ставок
Гринатом
Faraway
Timeweb
Findmykids
РСХБ-Интех
Kupsilla
CUSTIS
Яндекс
Yandex.Practicum
Muse Group
R-Vision
Serenity
X5 Tech
Exness
Garage Eight
Click
BrainRocket
EX CORP.
Avito
CMT
Алабуга
KnowledgeCity
P2P.ORG
Brainway
Linked Helper
VOLKA GAMES
X5 Digital
Hoodies
БАРС Груп
Яндекс Райдтех
Название скрыто (Ритейл)
UNIREST
MTS
Сбер
Название скрыто (Банк)
Emerging Travel Group
Fundraise Up
Okkam
МБС
Название скрыто (Финансовые технологии)
CONTENT AI
ITNTalents
Rocket Tech
GGSel
Яндекс Финтех
Yandex Cloud
Веб-Сервер
EdgeЦентр
Dif.tech (Plata)
Eqwile
Rusprofile
BI.ZONE
Kviku
Мир Plat.Form (команда НСПК)
CIAN
Mindbox
Яндекс (Поисковый п

In [ ]:
unique_salaries = set(vacancy.get('salary') for vacancy in vacancies if vacancy.get('salary') is not None)

print('\n'.join(unique_salaries))

104 400 —‍ 113 000 ₽/мес на руки
220 000 —‍ 350 000 ₽/мес на руки
160 000 —‍ 250 000 ₽/мес на руки
170 000 —‍ 190 000 ₽/мес на руки
250 000 —‍ 340 000 ₽/мес на руки
290 000 —‍ 325 000 ₽/мес на руки
от 225 000 ₽/мес на руки
190 000 —‍ 350 000 ₽/мес на руки
160 000 —‍ 680 000 ₽/мес на руки
200 000 —‍ 440 000 ₽/мес на руки
70 000 —‍ 90 000 ₽/мес на руки
170 000 —‍ 300 000 ₽/мес на руки
230 000 —‍ 300 000 ₽/мес на руки
от 4 400 €/мес на руки
от 280 000 ₽/мес на руки
180 000 —‍ 400 000 ₽/мес на руки
450 000 —‍ 550 000 ₽/мес на руки
170 000 —‍ 220 000 ₽/мес на руки
от 120 000 ₽/мес на руки
180 000 —‍ 300 000 ₽/мес на руки
383 000 —‍ 585 000 ₽/мес на руки
60 000 —‍ 100 000 ₽/мес на руки
400 000 —‍ 500 000 ₽/мес на руки
100 000 —‍ 250 000 ₽/мес на руки
220 000 —‍ 270 000 ₽/мес на руки
380 000 —‍ 450 000 ₽/мес на руки
450 000 —‍ 750 000 ₽/мес на руки
50 000 —‍ 70 000 ₽/мес на руки
от 4 900 €/мес на руки
1 500 —‍ 2 500 €/мес на руки
290 000 —‍ 400 000 ₽/мес на руки
от 360 000 ₽/мес на руки
от 70

In [ ]:
unique_locations = set()

for vacancy in vacancies:
    locations = vacancy.get('locations', [])
    unique_locations.update(locations)

print('\n'.join(unique_locations))

📍 Москва (м. Смоленская)
Санкт-Петербург (м. Маяковская)
📍 Москва (м. Румянцево)
📍 Новосибирск
📍 Москва (м. Технопарк)
Белгород
Новокузнецк
📍 Лимасcол (Кипр)
📍 Санкт-Петербург (м. Спортивная)
Санкт-Петербург (м. Владимирская)
📍 Пермь
📍 Москва (м. Арбатская)
📍 Москва (м. Краснопресненская)
📍 Москва (м. Охотный ряд / Площадь Революции / Театральная)
Испания
Владивосток
📍 Москва (м. Верхние Котлы)
📍 Никосия (Кипр)
Португалия
Майкоп
Иваново
📍 Лимасол (Кипр)
📍 Москва (м. Спортивная)
Краснодар
📍 Москва (м. Отрадное)
📍 Санкт-Петербург (м. Чернышевская)
Волгоград
📍 Санкт-Петербург (м. Беговая)
📍 Санкт-Петербург (м. Лиговский проспект)
📍 Москва (м. Белорусская)
📍 Малага (Испания)
Великий Новгород
Нижний Новогород
📍 Москва (Волгоградский проспект)
📍 Москва (м. Шаболовская)
📍 Москва (м. Третьяковская)
Санкт-Петербург (м. Невский проспект)
Санкт-Петербург (м. Площадь Александра Невского)
📍 Москва (м. Новослободская)
Белград (Сербия)
Санкт-Петербург (м. Новочеркасская)
📍 Нижний Новгород
📍 Москва (м

In [ ]:
locations = list(unique_locations)

def extract_city(location):
    return re.sub(r"📍\s*|\s*\(.*\)", "", location).strip()

cities = [extract_city(location) for location in locations]

print('\n'.join(set(cities)))

Белгород
Новокузнецк
Владивосток
Испания
Пафос
Португалия
Вильнюс
Майкоп
Иваново
Россия
Краснодар
Волгоград
Каймановы Острова
Великий Новгород
Нижний Новогород
Армения
Хабаровск
Белград
Липецк
Орел
Лимассол
София
Турция
Екатеринбург, Нижний Новгород
Уфа
Сочи
Лимасол
Сербия
Москва
Нижний Новгород
Пермь
Никосия
Тбилиси
Казань
Саратов
Астрахань
Ульм
Все города РФ, кроме Москвы
Беларусь
Лондон
Санкт-Петербург
Польша
Сочи, Самара
Екатеринбург
Ростов-на-Дону
Ереван
Зеленодольск
Калуга
Астана, Алматы
Европа
Берлин
Татарстан
Все города РФ
Казань, Новосибирск, Симферополь
Томск
ЕС
Самара
Алматы
Новосибирск
Череповец
Грузия
Красноярск
Ярославль
Тюмень
Мексика
Симферополь
Регионы РФ
Московская область
Воронеж
Малага
Иннополис
Омск
Вологда
Муром
Тула
Кипр
Иркутск
Екатеринбург, Нижний Новгород, Казань, Новосибирск, Симферополь
Казахстан
Лимасcол
Минск
Черногория


In [ ]:
unique_categories = set()

for vacancy in vacancies:
    categories = vacancy.get('categories', [])
    unique_categories.update(categories)

print('\n'.join(unique_categories))

Можно удалённо из РФ
Офис или гибрид
Офис несколько дней в неделю
Помощь с переездом
Самостоятельный переезд
Полная удалёнка


In [ ]:
unique_terms_keys = set()

for vacancy in vacancies:
    terms = vacancy.get('terms', {})
    unique_terms_keys.update(terms.keys())

print(unique_terms_keys)

{'Требуемый опыт', 'Уровень', 'Английский', 'Специализация'}


In [ ]:
unique_terms_values = {}

for vacancy in vacancies:
    terms = vacancy.get('terms', {})
    for key, value in terms.items():
        if key not in unique_terms_values:
            unique_terms_values[key] = set()
        unique_terms_values[key].add(value)

for key, values in unique_terms_values.items():
    print(f'{key}:')
    print('\n'.join(values), end='\n\n')

Специализация:
Frontend / СТО
Architect / Node.js
Android / iOS
Data Scientist / Data Engineering / Python
CTO / Architect / Data Engineering
System Analyst / Business Analyst
Python / JS/Frontend Developer
Product Analyst
HR
Design & UX Frontend
Go / Java / Python
Mobile (Android / iOS)
C++ / JavaScript
Infrastructure Engineer
JS / Frontend Developer
DevOps/Data Engineering
Data Scientist / Product Analyst
Data Science / C++
Data Science / Machine Learning
DevOps
С# / Architect
Java
Node.js / TypeScript
QA Auto
Product Manager
Project Manager / СТО
BusinessAnalyst / System Analyst
PHP / Go / JS
Frontend Developer
CTO / Architect / Go / Java / Python
Symfony / Vue.js
Backend Developer
С / С++
Engineering Management
Recruiter
Python / Go / Java
Product Analyst / Data Scientist
JS/Frontend Developer
QA Auto / Python / Java
Product Designer
C++/C#
Golang / PHP / Java
CTO / Project Manager / Data Engineering
Android
С++ / Java
Architect
Golang / Python
PHP/Kotlin/Java
.NET / React
JS / Fro

In [ ]:
unique_stack_items = set()

for vacancy in vacancies:
    stack = vacancy.get('stack', [])
    unique_stack_items.update(stack)

print('\n'.join(unique_stack_items))

File Server Failover Cluster
Charles Proxy
SaltStack/Ansible/Puppet
Robot Framework
1C:УТ
OpenWRT
RuPost/Postfix-Dovecot-SOGo/CommuniGate Pro/Exim/Zimbra/Exchange
UIKit
Volcano
Go/Java/Python
ASP.NЕТ
Retrofit
x86 Servers
Android Studio/XCode
ZooKeeper
OpenCV
Java
OneLogin
WPA
GPU
GitHub/Bitbucket
Go/Java/Python/PHP/C#
Git/TFS
Jamf
Ansible/Puppet/Terraform
Nmap
PostgreSQL/MySQL/MongoDB
Git / SVN
JetBrains IDEA
Dagster
Symfony 5.x-6.x
mmdeploy
Snowflake
OpenVPN
Ubuntu/Gentoo
Greenplum
Windows/Linux/MacOS
Charles/Fiddler/Proxyman
restic
JUnit
Secure SDLC
Python/Shell
DBMS
Zeplin
Python/R/SQL
YAML
C/С++
Feast
CMDB
Pinia
Selenium/Selenide
JSON/XML
DDL
React
Tarantool
Go/Python/Java
Firewall
HBase
PostgreSQL / Oracle / Microsoft SQL Server
Unix
Infrastructure
Transformers
AstraLinux
REST / SOAP
Android Architecture Components
Keras/PyTorch/TensorFlow
Enterprise Architect
Astra Linux/Debian/CentOS
Hive
Python/Go/JavaScript
Charles
MS SQL 19+
Argo CD
ArchiMate/C4
ELK
LangChain
Alias
Golang/PHP

In [ ]:
unique_descriptions_keys = set()

for vacancy in vacancies:
    descriptions = vacancy.get('descriptions', {})
    unique_descriptions_keys.update(descriptions.keys())

print('\n'.join(unique_descriptions_keys))

The tasks will include
Нам важно, что бы у вас было
Что мы можем тебе дать:
Обучение и карьера:
Что вас ждет
От кандидата ожидаем:
Что тебе может дать компания
Key responsibilities include
Kotlin-разработчик в нашей команде
What you will do
Что мы предлагаем для тебя:
Как построена работа
Структура собеседований
Этапы найма
Финансовые условия:
Какие задачи вас ждут.
Он
Мы ждём
Что необходимо для максимального результата
Что важно
Условия
В каждом проекте у нас есть основные направления
About the team
Для сотрудников компании
Из важного:
Формальный список того, что нужно делать
Ждем от тебя
Минимальные требования
Кого мы ищем
Ключевые навыки, программы, софт:
Что мы ждем:
Что мы ожидаем от кандидата
Что мы ожидаем от Вас
Требования к кандидатам
О компании
ЧЕМ ТЫ БУДЕШЬ ЗАНИМАТЬСЯ:
Навыки
Что ждём от кандидата:
Что в Точке необычного
Пожелания к опыту
Что ожидаем от тебя:
Иви ценит:
Fullstack-разработчик в команде
Ты наш человек, если у тебя есть
Примеры вопросов, на которые мы ищем отве

Вот какие данные из получившегося массива включим в датасет:

**`title`**</br>
Оставляем как есть (будет столбец `title`).


**`company`**</br>
Для большинства строк оставляем как есть, для тех где название скрыто делаем 'None' (будет столбец `company`).

**`salary`**</br>
Делаем два числовых столбца: `salary_from` и `salary_to`. Если указана одна зарплата, пишем ее в оба столбца. Для удобства сразу переведем в рубли, хотя бывает еще в долларах и в евро (по примерному курсу).

**`locations`**</br>
Делаем один строковый столбец: `locations`. Из каждого элемента массива уберем лишнее (эмодзи, значения в скобках), перечислим через запятую.

**`categories`**</br>
Категорий мало. Из них составим следующие признаки:

- `type`</br>
remote (если есть 'Полная удалёнка')</br>
hybrid (если есть 'Офис или гибрид', 'Офис несколько дней в неделю')</br>
office (в других случаях)

- `relocation`</br>
1 (если есть 'Самостоятельный переезд' / 'Помощь с переездом' и нет 'Можно удалённо из РФ')</br>
0 (в других случаях)

- `relocation_help`</br>
1 (если есть 'Помощь с переездом')</br>
0 (в других случаях)

**`terms`**</br>
Самих условий мало, а вот данные в них разные. В итоге, сделаем 4 столбца:

- `specialization`</br>
Оставляем текстом как есть (т.к. очень много значений, это будет текстовое поле).

- `level`</br>
Оставляем как есть, это отличный категориальный признак (6 вариантов + может быть неизвестно).

- `english_level`</br>
Оставляем только код (B1, ...), это тоже отличный категориальный признак (5 вариантов  + может быть неизвестно).

- `min_experience`</br>
Это минимальное число лет опыта, распарсим строки типа '3+ лет' в числа. Если не указано, это не NaN, а 0 (получается, опыт не требуется).

**`stack`**</br>
Вариантов очень много, так что сделаем тоже текстовое поле (через запятую, назовем `stack`).

**`descriptions`**</br>
К сожалению, оказалось что заголовки везде разные, и какой-то их порядок гаранировать нельзя. Поэтому в рамках работы просто соберем все в одно текстовое поле с описанием. Но вообще, это можно было бы использовать как-то обработав сами тексты, так что есть простор для дальнейших работ :) Назовем столбец `description`.

In [ ]:
def process_company(company):
  if company and 'Название скрыто' in company:
    return None

  return company

In [56]:
def process_salary(salary):
    # смотрим, что зарплата есть
    if pd.isna(salary) or salary == '':
        return None, None

    # убираем пробелы и ищем числа
    salary = str(salary).replace(' ', '')
    numbers = re.findall(r'\d+', salary)
    numbers = [int(num) for num in numbers]

    # должно получиться 1 или 2 числа: на всякий случай проверяем
    if len(numbers) not in [1, 2]:
        print(f'error: {salary}')
        return None, None

    # получаем валюту
    if '₽' in salary:
        currency = '₽'
    elif '€' in salary:
        currency = '€'
    elif '$' in salary:
        currency = '$'
    else:
        currency = None

    # переводим в рубли
    if currency == '€':
        numbers = [num * 93 for num in numbers]
    elif currency == '$':
        numbers = [num * 89 for num in numbers]
    elif currency is None:
        return None, None

    # задаем зарплаты от и до
    if len(numbers) == 1:
        salary_from = salary_to = numbers[0]
    else:
        salary_from, salary_to = numbers

    return salary_from, salary_to

In [ ]:
def process_locations(locations):
    # смотрим, что места есть
    if not locations:
        return None

    # убираем лишнее и пишем в строку через запятую
    return ', '.join([re.sub(r"📍\s*|\s*\(.*\)", "", location).strip() for location in locations])

In [ ]:
def process_categories(categories):
    # задаем дефолтные значения
    values = {
        'type': 'office',
        'relocation': 0,
        'relocation_help': 0
    }

    # если категорий вообще нет, возвращаем дефолт
    if not categories:
        return values

    # проверяем категории для типа работы
    if 'Офис или гибрид' in categories or 'Офис несколько дней в неделю' in categories:
      values['type'] = 'hybrid'

    if 'Полная удалёнка' in categories:
      values['type'] = 'remote'

    # проверяем категории для обязательной релокации
    if ('Самостоятельный переезд' in categories or 'Помощь с переездом' in categories) and (not 'Можно удалённо из РФ' in categories):
      values['relocation'] = 1

    # проверяем категории для наличия помощи с релокацией
    if 'Помощь с переездом' in categories:
      values['relocation_help'] = 1

    return values

In [84]:
def process_terms(terms):
    return {
        'specialization': terms.get('Специализация') if terms.get('Специализация') else None, # если есть оставляем
        'level': terms.get('Уровень') if terms.get('Уровень') else None, # если есть оставляем
        'english_level': terms.get('Английский').split()[0] if terms.get('Английский') else 'Не требуется', # если есть оставляем первое слово (код)
        'min_experience': int(re.findall(r'\d+', terms.get('Требуемый опыт', '0'))[0]) if terms.get('Требуемый опыт') else 0 # если есть парсим, иначе 0
    }

Преобразуем получившийся массив в датафрейм:

In [96]:
vacancies_copy = [vacancy.copy() for vacancy in vacancies]

for vacancy in vacancies_copy:
    # компания
    vacancy['company'] = process_company(vacancy.get('company'))

    # зарплата
    salary_from, salary_to = process_salary(vacancy.get('salary', ''))
    vacancy['salary_from'] = salary_from
    vacancy['salary_to'] = salary_to

    # локация
    vacancy['locations'] = process_locations(vacancy.get('locations', []))

    # Обрабатываем категории
    categories = process_categories(vacancy.get('categories', []))
    vacancy.update(categories)

    # условия
    terms = process_terms(vacancy.get('terms', {}))
    vacancy.update(terms)

    # стек
    vacancy['stack'] = ', '.join(vacancy.get('stack', []))

    # описание
    vacancy['description'] = '\n\n'.join(vacancy.get('descriptions', {}).values()) if isinstance(vacancy.get('descriptions'), dict) else ''


get_match_df = pd.DataFrame(vacancies_copy)
get_match_df = get_match_df[[
    'title',
    'company',
    'salary_from',
    'salary_to',
    'locations',
    'type',
    'relocation',
    'relocation_help',
    'specialization',
    'level',
    'english_level',
    'min_experience',
    'stack',
    'description'
]]

get_match_df.head()

,title,company,salary_from,salary_to,locations,type,relocation,relocation_help,specialization,level,english_level,min_experience,stack,description
0,Senior DevOps/DevSecOps Engineer,Collectly,534000.0,712000.0,"Бразилия, Сербия, Аргентина, Черногория, Испан...",remote,0,0,DevOps / Information Security,Senior,B2,0,"Ansible, Terraform, CI/CD, AWS/Azure/GCP, Jenk...","Design, develop, and maintain robust infrastru..."
1,Разработчик на Python (Непрерывная интеграция ...,Яндекс (Поисковый портал),220000.0,450000.0,Москва,hybrid,0,0,Python,Middle,Не требуется,3,"Python, CI/CD",Стремитесь выбирать хорошие архитектурные реше...
2,Ведущий продакт-менеджер экосистемы по взаимод...,Avito,355000.0,466000.0,Москва,remote,0,0,Product Manager,Senior,Не требуется,0,,Разработка и внедрение стратегии развития CRM-...
3,Архитектор продукта (1С: Управление проектами),Гринатом,300000.0,300000.0,Москва,hybrid,0,0,1C,Middle,Не требуется,3,"1С: Предприятие 8, 1С:PM Управление проектами ...",Быть единой точкой архитектурных решений за ко...
4,Affiliate Manager,Winline,150000.0,200000.0,Москва,hybrid,0,0,Product Analyst / Product Manager / Project Ma...,Middle,Не требуется,2,"Google Analytics, Яндекс.Метрика, Excel, Power...",Поддерживать и развивать партнерские отношения...


In [97]:
get_match_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            1200 non-null   object 
 1   company          1132 non-null   object 
 2   salary_from      1137 non-null   float64
 3   salary_to        1137 non-null   float64
 4   locations        891 non-null    object 
 5   type             1200 non-null   object 
 6   relocation       1200 non-null   int64  
 7   relocation_help  1200 non-null   int64  
 8   specialization   1200 non-null   object 
 9   level            1200 non-null   object 
 10  english_level    1200 non-null   object 
 11  min_experience   1200 non-null   int64  
 12  stack            1200 non-null   object 
 13  description      1200 non-null   object 
dtypes: float64(2), int64(3), object(9)
memory usage: 131.4+ KB


In [98]:
get_match_df.isna().sum()

,0
title,0
company,68
salary_from,63
salary_to,63
locations,309
type,0
relocation,0
relocation_help,0
specialization,0
level,0


In [105]:
# создаем новый salary, который будет средним из salary_from и salary_to
get_match_df["salary"] = get_match_df[["salary_from", "salary_to"]].mean(axis=1, skipna=True)
get_match_df.head()

,title,company,salary_from,salary_to,locations,type,relocation,relocation_help,specialization,level,english_level,min_experience,stack,description,salary
0,Senior DevOps/DevSecOps Engineer,Collectly,534000.0,712000.0,"Бразилия, Сербия, Аргентина, Черногория, Испан...",remote,0,0,DevOps / Information Security,Senior,B2,0,"Ansible, Terraform, CI/CD, AWS/Azure/GCP, Jenk...","Design, develop, and maintain robust infrastru...",623000.0
1,Разработчик на Python (Непрерывная интеграция ...,Яндекс (Поисковый портал),220000.0,450000.0,Москва,hybrid,0,0,Python,Middle,Не требуется,3,"Python, CI/CD",Стремитесь выбирать хорошие архитектурные реше...,335000.0
2,Ведущий продакт-менеджер экосистемы по взаимод...,Avito,355000.0,466000.0,Москва,remote,0,0,Product Manager,Senior,Не требуется,0,,Разработка и внедрение стратегии развития CRM-...,410500.0
3,Архитектор продукта (1С: Управление проектами),Гринатом,300000.0,300000.0,Москва,hybrid,0,0,1C,Middle,Не требуется,3,"1С: Предприятие 8, 1С:PM Управление проектами ...",Быть единой точкой архитектурных решений за ко...,300000.0
4,Affiliate Manager,Winline,150000.0,200000.0,Москва,hybrid,0,0,Product Analyst / Product Manager / Project Ma...,Middle,Не требуется,2,"Google Analytics, Яндекс.Метрика, Excel, Power...",Поддерживать и развивать партнерские отношения...,175000.0


In [104]:
from google.colab import files
get_match_df.to_csv('get_match_df.csv', index=False)
files.download('get_match_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>